# Factorisation LU

Mettez ci-dessous les imports classiques de librairie Python

In [2]:
%matplotlib inline
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

M = np.load('TP6_Matrice.npy')

Le but de ce TP est d'implémenter la factorisation LU, avec et sans permutations, et de tester ces méthodes pour résoudre un système linéaire. Dans cette dernière partie, on implémentera un algorithme de résolution de systèmes triangulaires.

## 1) Algorithme de factorisation LU standard

On se propose tout d'abord d'implémenter un code de factorisation LU qui prendra en entrée une matrice $A$ carré de taille quelconque et qui donnera en sortie le couple de matrice L et U correspondant à la factorisation LU de la matrice A. On rappelle que la factorisation LU s'obtient à partir de la méthode de pivot de Gauss. Plus précisément, la factorisation LU peut se résumer à l'algorithme suivant

$$
A^{(0)} = A\in M_n(\mathbb{R}),\\
L^{(0)} = 0 \in M_n(\mathbb{R}),\\
\textrm{Pour}\; k = 0,1,2,\cdots,n-1:\;
\left\{\begin{array}{ll}
\ell^{(k)} = \left\{\begin{array}{ll}
0,\;\textrm{pour}\;1\leq i\leq k-1,\\
\frac{a_{i,k}^{(k)}}{a_{k,k}^{(k)}},\;\textrm{pour}\;k\leq i\leq n,
\end{array}\right.\\
L^{(k+1)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_i,\; \textrm{pour}\; j = k\;\textrm{et}\; k\leq i\leq n,
\\ L^{(k)}_{i,j}\; \textrm{sinon},
\end{array}\right.\\
B^{(k)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_{i} a^{(k)}_{k,j},\; \textrm{pour}\; k+1\leq i\leq n \;\textrm{et}\; k\leq j\leq n,
\\ 0\; \textrm{sinon},
\end{array}\right.,\\
A^{(k+1)} = A^{(k)} - B^{(k)},\\
\end{array}\right.
$$

où
- $\ell^{(k)}$ est un vecteur de $\mathbb{R}^n$,
- $A^{(k)}$ est une matrice de $M_n(\mathbb{R})$ dont on note les coefficients $(a^{(k)}_{i,j})_{1\leq i,j\leq n}$.

A la fin de cette algorithme, on obtient $L = L^{(n-1)}$ et $U = A^{(n-1)}$.

> **A faire :** Implémenter la fonction `LU` qui permet de calculer la factorisation LU d'une matrice $A$ carré de dimension quelconque à partir de l'algorithme précédent. On prendra soin de vérifier si le pivot est non-nul et, dans le cas contraire, on renverra une erreur à l'utilisateur. On pourra tester la fonction sur la matrice
$$
A = \left(\begin{array}{ccc}
2 & 1 & 1 \\
6 & 2 & 1 \\
-2 & 1 & 1
\end{array}\right),
$$
dont la décomposition LU est donnée par
$$
L = \left(\begin{array}{ccc}
1 & 0 & 0 \\
3 & 1 & 0 \\
-1 & -3 & 1
\end{array}\right)\quad\textrm{et}\quad U = \left(\begin{array}{ccc}
2 & 1 & 1 \\
0 & -1 & -2 \\
0 & 0 & -4
\end{array}\right).
$$

In [98]:
for k in range (0,n):
        for i in range (0,n):
            if(i<k-1):
                lk = 0
            else:
                if(A[k][k] == 0):
                    print('Erreur')
                    return
                else:
                    lk = A[i][k]/A[k][k]

            L[i][k] = lk

        B[i][k] = lk*A[k][i]

        A = A - B

        print(lk)
        print(A)
        
         n = np.size(A[0])
    L = np.zeros((n, n))
    ListEinv = []
    
    E = np.identity(n)
    for k in range(0,n):
        for i in range(1,n):
            print(A[i][k])
            print(A[k][k])
            lik = A[i][k]/float(A[k][k])
            E[i][k] = -lik
            print(lik)

        ListEinv.append(npl.inv(E))
        print(E)
        A = np.dot(E,A)
        print(A)
            
    L = ListEinv[0]
    for j in range(1,n):
        L = L*ListEinv[j]

IndentationError: unexpected indent (<ipython-input-98-ec59b9f6c22a>, line 23)

In [3]:
def LU(A):
    n = np.size(A[0])
    L = np.zeros((n, n))
    M = np.copy(A)
    for k in range (0,n):
        l = np.zeros(n)
        for i in range(k, n):
            l[i] = M[i][k]/M[k][k]
        L[:,k] = l
        B = np.zeros((n,n))
        for j in range(k+1, n):
            for i in range(k, n):
                    B[j,i] = l[j] * M[k,i]
        M = M - B
    
    return L,M

A = np.array([[2,1,1],[6,2,1],[-2,2,1]])
L,U = LU(A)
print("L=",L)
print("U=",U)


L= [[ 1.  0.  0.]
 [ 3.  1.  0.]
 [-1. -3.  1.]]
U= [[ 2.  1.  1.]
 [ 0. -1. -2.]
 [ 0.  0. -4.]]


In [112]:
A = np.array([[2,1,1],[6,2,1],[-2,2,1]])
L,U = LU(A)
print("L=",L)
print("U=",U)

L= [[ 1.  0.  0.]
 [ 3.  1.  0.]
 [-1. -3.  1.]]
U= [[ 2.  1.  1.]
 [ 0. -1. -2.]
 [ 0.  0. -4.]]


## 2) Algorithme de factorisation LU avec stratégie de pivot partiel

On passe maintenant à la factorisation LU avec la stratégie de pivot partiel. Cette méthode met en place, durant le pivot de Gauss, des permutations sur les lignes qui permettent de diminuer les erreurs numériques et de parer au problème d'un pivot nul. Ces permutations sur les lignes vont faire en sorte que le pivot dans la méthode de Gauss est toujours le plus grand possible. En pratique, à chaque étape $k$, lors du choix du pivot, on va chercher dans la colonne $k$ le plus grand pivot (en valeur absolue) puis on va procèder à une permutation entre la ligne de ce pivot et la ligne $k$. Plus précisément, la factorisation LU avec permutations peut se résumer à l'algorithme suivant

$$
A^{(0)} = A\in M_n(\mathbb{R}),\\
L^{(0)} = 0\in M_n(\mathbb{R}),\\
P^{(0)} = \textrm{Id}\in M_n(\mathbb{R}),\\
\textrm{Pour}\; k = 0,1,2,\cdots,n-1:\;
\left\{\begin{array}{ll}
m = \textrm{argmax}_{k\leq j \leq n} |a_{j,k}^{(k)}|\\
Q^{k,m} = \textrm{Matrice de permutation des indices $k$ et $m$}\\
P^{(k+1)} = Q^{k,m}P^{(k)},\\
L^{(k+1/2)} = Q^{k,m}L^{(k)},\\
A^{(k+1/2)} = Q^{k,m}A^{(k)},\\
\ell^{(k)} = \left\{\begin{array}{ll}
0,\;\textrm{pour}\;1\leq i\leq k-1,\\
\frac{a_{i,k}^{(k+1/2)}}{a_{k,k}^{(k+1/2)}},\;\textrm{pour}\;k\leq i\leq n,
\end{array}\right.\\
L^{(k+1)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_i,\; \textrm{pour}\; j = k\;\textrm{et}\; k+1\leq i\leq n,
\\ L^{(k+1/2)}_{i,j}\; \textrm{sinon},
\end{array}\right.\\
B^{(k)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_{i} a^{(k+1/2)}_{k,j},\; \textrm{pour}\; k+1\leq i\leq n \;\textrm{et}\; k\leq j\leq n,
\\ 0\; \textrm{sinon},
\end{array}\right.,\\
A^{(k+1)} = A^{(k+1/2)} - B^{(k)},\\
\end{array}\right.
$$

A la fin de cette algorithme, on obtient $L = L^{(n-1)}$, $U = A^{(n-1)}$ et $P = P^{(n-1)}$. La matrice de permutation n'a pas vraiment besoin d'être assemblée, on pourra simplement écrire une fonction `Permut` qui prendra en argument une matrice et le couple d'indice et donnera en sortie la matrice avec ses lignes permutées. Afin d'identifier l'indice du plus grand pivot, on pourra s'aider de la fonction `argmax`. On rappelle de plus que, pour une matrice de permutation $P$, on a

$$
P^{-1} = P^T.
$$

> **A faire :** Implémenter la fonction `PLU` qui permet de calculer la factorisation LU d'une matrice $A$ carré de dimension quelconque à partir de l'algorithme précédent. On prendra soin de vérifier si le pivot est non-nul et, dans le cas contraire, on renverra une erreur à l'utilisateur. On pourra tester la fonction sur la matrice
$$
A = \left(\begin{array}{ccc}
1 & 1 & 1 & 1 \\
1 & 1 & 3 & 3 \\
1 & 1 & 2 & 3\\
1 & 3 & 3 & 3
\end{array}\right),
$$
dont la décomposition PLU est donnée par
$$
P = \left(\begin{array}{ccc}
1 & 0 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0
\end{array}\right),\quad L = \left(\begin{array}{ccc}
1 & 0 & 0 & 0\\
1 & 1 & 0 & 0\\
1 & 0 & 1 & 0\\
1 & 0 & 1/2 & 1
\end{array}\right)\quad\textrm{et}\quad U = \left(\begin{array}{ccc}
1 & 1 & 1 & 1\\
0 & 2 & 2 & 2\\
0 & 0 & 2 & 2\\
0 & 0 & 0 & 1
\end{array}\right).
$$

In [213]:
def Permut(A,i,j):
    ligne_A_temporaire = np.copy(A[i,:])    
    A[i,:] = A[j, :]
    A[j, :] = ligne_A_temporaire
    return A

def PLU(A):
    n = np.size(A[0])
    L = np.zeros((n, n))
    M = np.copy(A)
    P = np.eye(n)
    for k in range (0,n):
        l = np.zeros(n)
        #Recherche de maximum dans la colonne
        pmax = np.amax(M[k:,k])
        #print(M[k:,k])
        j = k
        while ( pmax != M[j, k]):
            j = j + 1
        
        M = Permut(M, j, k)
        P = Permut(P, j, k)
        L = Permut(L, j, k)
        
        for i in range(k, n):    
            l[i] = M[i][k]/M[k][k]
            
        L[:,k] = l
        B = np.zeros((n,n))
        
        for j in range(k+1, n):
            for i in range(k, n):
                    B[j,i] = l[j] * M[k,i]
        #print(k)
        #print(M)
        M = M - B
        
    
    return P,L,M

A = np.array([[1,1,1,1],[1,1,3,3],[1,1,2,3],[1,3,3,3]])
P,L,U = PLU(A)
print("P=",P)
print("L=",L)
print("U=",U)

P= [[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]]
L= [[ 1.   0.   0.   0. ]
 [ 1.   1.   0.   0. ]
 [ 1.   0.   1.   0. ]
 [ 1.   0.   0.5  1. ]]
U= [[ 1.  1.  1.  1.]
 [ 0.  2.  2.  2.]
 [ 0.  0.  2.  2.]
 [ 0.  0.  0.  1.]]


In [208]:
A = np.array([[1,1,1,1],[1,1,3,3],[1,1,2,3],[1,3,3,3]])
P,L,U = PLU(A)
print("P=",P)
print("L=",L)
print("U=",U)

0
3
3
3
P= [[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  1.  1.]]
L= [[ 1.   0.   0.   0. ]
 [ 1.   1.   0.   0. ]
 [ 1.   0.   1.   0. ]
 [ 1.   0.   0.5  1. ]]
U= [[ 1.  1.  1.  1.]
 [ 0.  2.  2.  2.]
 [ 0.  0.  2.  2.]
 [ 0.  0.  0.  1.]]


## 3) Résolution de systèmes triangulaires et comparaison des méthodes

Nous allons à présent comparer les 2 méthodes que nous venons d'implémenter. Le but est de tester ces méthodes sur un système linéaire. Or, on sait que pour résoudre un tel système il faut passer par la résolution de systèmes triangulaires (supérieurs et inférieurs). Ainsi, avant de passer à la suite, on se propose d'implémenter une méthode permettant de résoudre un système triangulaire, qu'il soit inférieur ou supérieur.

On rappelle que l'algorithme de résolution pour un système triangulaire de la forme

$$
Tx = b,
$$

s'écrit, si $T$ est triangulaire inférieure,

$$
x_1 = b_1/T_{1,1}\\
\textrm{Pour}\; k = 2,3,\cdots, n\; :
x _k = \left(b_k - \sum_{j = 1}^{k-1} T_{k,j} x_j\right)/T_{k,k},
$$

et, si $T$ est triangulaire supérieure,

$$
x_n = b_n/T_{n,n}\\
\textrm{Pour}\; k = n-1,n-2,\cdots, 1\; :
x _k = \left(b_k - \sum_{j = k+1}^{n} T_{k,j} x_j\right)/T_{k,k}.
$$

> **A faire :** Implémenter la fonction `Solve_Triang` qui permet de résoudre un système linéaire triangulaire (inférieur ou supérieur). Cette fonction prendra en entrée une matrice et un vecteur (correspondant aux données du système linéaire) ainsi qu'une variable précisant si le système est triangulaire supérieur ou inférieur et donnera en sortie un vecteur qui est la solution que l'on cherche.

In [ ]:
def Solve_Triang(T,b,s = 0):
    
    return x

On a charger, en début de ce notebook, la matrice $M$ de taille 100. A l'aide des deux méthodes de factorisation et de la méthode de résolution d'un système triangulaire, vous allez résoudre le système linéaire

$$
Mx = b,
$$

où $b\in\mathbb{R}^{100}$ est tel que $b_i = 1$, pour tout $1\leq i\leq n$.

> **A faire :** Résoudre le système précédent à l'aide de la factorisation LU standard et celle avec permutations. Comparer les résultats obtenus avec la solution obtenue avec `npl.solve(M,b)`. Quelle méthode donne le meilleur résultat?